Mempersiapkan data latih dan data uji

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# function untuk konversi dari ubyte menjadi file TXT
def convert(img_file, label_file, txt_file, n_images):
  print("\nOpening binary pixels and labels files ")
  lbl_f = open(label_file, "rb")   # labels (digits)
  img_f = open(img_file, "rb")     # pixel values
  print("Opening destination text file ")
  txt_f = open(txt_file, "w")      # output to write to

  print("Discarding binary pixel and label headers ")
  img_f.read(16)   # discard header info
  lbl_f.read(8)    # discard header info

  print("\nReading binary files, writing to text file ")
  print("Format: 784 pixels then labels, tab delimited ")
  for i in range(n_images):   # number requested 
    lbl = ord(lbl_f.read(1))  # Unicode, one byte
    for j in range(784):  # get 784 pixel vals
      val = ord(img_f.read(1))
      txt_f.write(str(val) + "\t") 
    txt_f.write(str(lbl) + "\n")
  img_f.close(); txt_f.close(); lbl_f.close()
  print("\nDone ")

In [6]:
convert("dataset/train-images.idx3-ubyte", "dataset/train-labels.idx1-ubyte", "dataset/trainSet.txt", 60000)
convert("dataset/t10k-images.idx3-ubyte", "dataset/t10k-labels.idx1-ubyte", "dataset/testSet.txt", 10000)


Opening binary pixels and labels files 
Opening destination text file 
Discarding binary pixel and label headers 

Reading binary files, writing to text file 
Format: 784 pixels then labels, tab delimited 

Done 

Opening binary pixels and labels files 
Opening destination text file 
Discarding binary pixel and label headers 

Reading binary files, writing to text file 
Format: 784 pixels then labels, tab delimited 

Done 


Membuat Data Pipeline

In [2]:
import torch
import numpy as np

class MNIST_Dataset(torch.utils.data.Dataset):
  # 784 tab-delim pixel values (0-255) then label (0-9)
  def __init__(self, src_file, transform = None):
    all_xy = np.loadtxt(src_file, usecols=range(785),
      delimiter="\t", comments="#", dtype=np.float32)

    self.transform = transform
    tmp_x = all_xy[:, 0:784]  # all rows, cols [0,783]
    tmp_x /= 255
    tmp_x = tmp_x.reshape(-1, 1, 28, 28)
    tmp_y = all_xy[:, 784]

    self.x_data = \
      torch.tensor(tmp_x, dtype=torch.float32)
    self.y_data = \
      torch.tensor(tmp_y, dtype=torch.int64)
     

  def __len__(self):
    return len(self.x_data)

  def __getitem__(self, idx):
    lbl = self.y_data[idx]  # no use labels
    pixels = self.x_data[idx]

    if self.transform:
      pixels = self.transform(pixels) 
    return (pixels, lbl)

In [9]:
from torch.utils.data import DataLoader
from torchvision import transforms

# compose = transforms.Compose([
#     transforms.ColorJitter(brightness=0.5, contrast=0.2),
#     transforms.Normalize(mean=(0.1307), std=(0.3079)),
# ])

train_ds = MNIST_Dataset('dataset/trainSet.txt', transform=None)
generator1 = torch.Generator().manual_seed(42)
train_data, val_data = torch.utils.data.random_split(train_ds, [0.9, 0.1], generator=generator1)

val_loader = DataLoader(val_data, batch_size=32, shuffle=True)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

Mendefinisikan Network CNN

In [16]:
import torch.nn as nn
import torch
from torchvision import transforms

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=3, stride=1, padding=0), #[1, 28, 28] -> [8, 26, 26]
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.MaxPool2d(kernel_size = 3, stride = 2), #[8, 26, 26] -> [8, 12, 12] 
            nn.Dropout(0.25))

        self.layer2 = nn.Sequential(
            nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=0), #[8, 12, 12] -> [16, 10, 10]
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(kernel_size=3, stride=2) #[16, 10, 10] -> [16, 4, 4]
        )

        self.classifier = nn.Sequential(
            nn.Linear(256, 50),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(50, 10)
        )

        self.gradient = None
        self.relu = nn.ReLU()
        self.resize = transforms.Resize(28)
        self.flatten = nn.Flatten()

    def forward(self, x):
       z = self.layer1(x)
       z = self.layer2(z)
       h = z.register_hook(self.hook_activation)
       z = self.flatten(z)
       z = self.classifier(z)
       return z
    
    def get_activations(self, x):
        with torch.no_grad():
            x = self.layer1(x)
            return self.layer2(x)
        
    def hook_activation(self, grad):
        self.gradient = grad
    
modifiedNet = Net()

Training Loop

In [12]:
# early stopping callback
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = float('inf')

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [17]:
from tqdm import tqdm
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(modifiedNet.parameters(), lr=1e-4)
early_stopper = EarlyStopper(patience=3, min_delta=0.05)

epochs = 10
for epoch in range(epochs):
    running_loss, running_accuracy = 0.0, 0.0
    loop = tqdm(train_loader)
    for i, data in enumerate(loop, 0):
        loop.set_description(f"Epoch [{epoch+1}/{epochs}]")
        inputs, labels = data
        inputs, labels = inputs, labels

        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = modifiedNet(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        with torch.no_grad():
            batch_acc = torch.sum(torch.argmax(outputs, dim=1) == labels) / labels.shape[0]
            running_accuracy += batch_acc.item()
            running_loss += loss.item()
        loop.set_postfix(loss = running_loss / (i+1), accuracy = (running_accuracy / (i+1)), refresh = True)
    
    running_val_loss = 0.0
    val_loss = 0.0
    for i, data in enumerate(val_loader):
        inputs, labels = data
        outputs = modifiedNet(inputs)
        loss = criterion(outputs, labels)
        running_val_loss += loss.item()
        val_loss = running_val_loss / (i+1)
    print(f"Validation loss: {val_loss}")
    if early_stopper.early_stop(val_loss):
        break
    
          

Epoch [1/10]: 100%|██████████| 1688/1688 [00:14<00:00, 112.95it/s, accuracy=0.635, loss=1.13]


Validation loss: 0.5951412884478874


Epoch [2/10]: 100%|██████████| 1688/1688 [00:14<00:00, 114.61it/s, accuracy=0.864, loss=0.457]


Validation loss: 0.3750975030533811


Epoch [3/10]: 100%|██████████| 1688/1688 [00:14<00:00, 113.98it/s, accuracy=0.905, loss=0.32] 


...

Validation loss: 0.17146968486857542


Epoch [9/10]: 100%|██████████| 1688/1688 [00:14<00:00, 116.47it/s, accuracy=0.953, loss=0.159]


Validation loss: 0.16252754884276618


Epoch [10/10]: 100%|██████████| 1688/1688 [00:14<00:00, 114.60it/s, accuracy=0.956, loss=0.151]


Validation loss: 0.1613413555587226


In [18]:
torch.save(modifiedNet.state_dict(), "models/Model.pt")